In [2]:
from pynq import Overlay, allocate
overlay = Overlay("./cordic.bit")
cordiccart2pol = overlay.cordiccart2pol_0
cordiccart2pol.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, RESERVED_3=0, RESERVED_4=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  x = Register(x=write-only),
  y = Register(y=write-only),
  r = Register(r=0),
  r_ctrl = Register(r_ap_vld=0, RESERVED=0),
  theta = Register(theta=0),
  theta_ctrl = Register(theta_ap_vld=0, RESERVED=0)
}

In [3]:
import random
import cmath
import struct
import binascii
import numpy as np
r_error=np.arange(0,10,0.1)
theta_error = np.arange(0,10,0.1)
for i in range(1,2):

    x = random.uniform(-1,1)
    y = random.uniform(-1,1)
#计算golden output
    cn = complex(x,y)  #复数类型书写
    cn = cmath.polar(cn)
#将输入转换成byte发送给FPGA
    x = (struct.unpack('<I',struct.pack('<f',x))[0])
    y = (struct.unpack('<I',struct.pack('<f',y))[0])
    #向FPGA中写数值
    cordiccart2pol.write(0x10,x)
    cordiccart2pol.write(0x18,y)
    #启动
    cordiccart2pol.write(0x00,1)
    cordiccart2pol.write(0x00,0)
    r = hex(cordiccart2pol.read(0x20))
    r = r[2:]
    theta = hex(cordiccart2pol.read(0x30))
    theta = theta[2:]
#转换成float类型
    if r!= 0:
        r=struct.unpack('>f',binascii.unhexlify(r))
        r = r[0]
    if theta!=0:
        theta= struct.unpack('>f',binascii.unhexlify(theta))
        theta=theta[0]
#和golden输出进行比较
    r_error[i] = "{0:.6f}".format(abs(r-cn[0]))
    theta_error[i] = "{0:.6f}".format(abs(theta-cn[1]))
sum_sq_r=0
sum_sq_theta=0
for i in range(1,2):
    sum_sq_r = sum_sq_r +(r_error[i] * r_error[i])
    r_rnse=np.sqrt(sum_sq_r/(i+1))
    sum_sq_theta = sum_sq_theta + (theta_error[i] * theta_error[i])
    theta_rnse = np.sqrt(sum_sq_theta/(i+1))
    print("Radius RMSE:", r_rnse, "Theta RMSE", theta_rnse)
    if r_rnse<0.001 and theta_rnse < 0.001:
        print("PASS")
    else:
        print("FAIL")

Radius RMSE: 5.939696961966999e-05 Theta RMSE 2.050609665440988e-05
PASS
